In [1]:
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 

In [2]:
from utilities import make_model
class feat_test_mdoel(make_model):
    def __init__(self , name , clf , gamma ,x ,y , train_data_loc):
        self.features = x.columns.to_list() 
        self.train_data_loc = train_data_loc
        make_model.__init__(self , name , clf , gamma , x ,y)

In [3]:
labels = ['AGN' , 'STAR' , 'YSO' , 'HMXB' , 'LMXB' , 'ULX' , 'CV' , 'PULSAR']
scores = ['precision' , 'recall' , 'f1_score']
models = ['all_feat_v1_0' ,'no_color_0' , 'no_gal_coord_0' , 'no_IR_0' , 'no_mw_0' , 'no_optical_uv_0' ]
for m in models:
    model = joblib.load('models/no_color_0.pkl')

NameError: name 'joblib' is not defined

In [2]:
labels = ['AGN' , 'STAR' , 'YSO' , 'HMXB' , 'LMXB' , 'ULX' , 'CV' , 'PULSAR']
scores = ['rep']

In [4]:
import joblib 
model = joblib.load('models/all_feat_v1_0.pkl')
model.result['class_scores'].loc[labels].to_csv('temp.csv')

In [15]:
all_feat = {
    'AGN' : {
        'precision' : [] , 
        'recall' : [],
        'f1_score' : []
    } ,
    'YSO' :{
        'precision' : [] , 
        'recall' : [],
        'f1_score' : []
    },
    'STAR' :{
        'precision' : [] , 
        'recall' : [],
        'f1_score' : []
    },
    'HMXB' :{
        'precision' : [] , 
        'recall' : [],
        'f1_score' : []
    },
    'LMXB' :{
        'precision' : [] , 
        'recall' : [],
        'f1_score' : []
    } , 
    'ULX' :{
        'precision' : [] , 
        'recall' : [],
        'f1_score' : []
    },
    'CV':{
        'precision' : [] , 
        'recall' : [],
        'f1_score' : []
    },
    'PULSAR' :{
        'precision' : [] , 
        'recall' : [],
        'f1_score' : []
    }
}
for i in range(5):
    model = joblib.load(f'models/no_optical_uv_{i}.pkl')
    all_feat['AGN']['precision'].append(model.result['class_scores']['precision_score']['AGN'])
    all_feat['AGN']['recall'].append(model.result['class_scores']['recall_score']['AGN'])
    all_feat['AGN']['f1_score'].append(model.result['class_scores']['f1_score']['AGN'])

In [16]:
all_mean = {}
for l in labels:
    for s in scores:
        all_mean[l] = {}
        all_mean[l][s] = np.mean(all_feat[l][s])
all_mean

/home/kumaran/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/kumaran/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'AGN': {'f1_score': 0.9626410544584327},
 'STAR': {'f1_score': nan},
 'YSO': {'f1_score': nan},
 'HMXB': {'f1_score': nan},
 'LMXB': {'f1_score': nan},
 'ULX': {'f1_score': nan},
 'CV': {'f1_score': nan},
 'PULSAR': {'f1_score': nan}}